In [ ]:
import numpy as np
import pandas as pd
from model_tuning import tune,  conformal_predict
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor



df_model = pd.read_csv("dataset/hdb_final_dataset.csv")


/var/folders/7j/kvhbym5j0q9499lwlx3p8hh80000gn/T/ipykernel_84481/3204934396.py:11: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_model = pd.read_csv("../data/hdb_final_dataset.csv")


In [2]:
df_model.describe()

,floor_area_sqm,lease_commence_date,remaining_lease,adjusted_resale_price,adj_resale_price_per_sqm,latitude,longitude,min_dist_sch,min_dist_cbd,min_dist_mrt,storey_median,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION
count,238802.000000,238802.000000,238802.000000,2.388020e+05,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000,238802.00000,238802.000000,238802.000000,238802.000000,238802.000000,238802.000000
mean,96.961016,1995.410290,73.969732,5.616875e+05,5849.807495,1.367593,103.840892,1.775027,12.068535,0.909899,8.665422,0.000377,0.01745,0.242552,0.420604,0.245057,0.073605,0.000356
std,24.055285,13.850938,13.649046,1.865108e+05,1521.092931,0.042836,0.071430,1.311596,4.358681,0.781244,5.852919,0.019410,0.13094,0.428627,0.493657,0.430122,0.261127,0.018863
min,31.000000,1966.000000,40.000000,1.568250e+05,2340.671642,1.270380,103.685228,0.049585,0.721959,0.014630,2.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.000000,1985.000000,63.000000,4.248649e+05,4844.023989,1.337020,103.778110,0.856899,9.291717,0.431211,5.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,93.000000,1996.000000,74.000000,5.279775e+05,5516.060518,1.366806,103.846014,1.478137,12.983033,0.694613,8.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,112.000000,2006.000000,86.000000,6.587971e+05,6447.773656,1.395723,103.898879,2.268127,15.129038,1.088660,11.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000
max,366.700000,2021.000000,97.000000,1.619760e+06,16017.509677,1.457071,103.987805,7.581888,19.757794,4.837718,50.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
from sklearn.model_selection import train_test_split
exog = ['remaining_lease', 'min_dist_sch', 'storey_median', 'min_dist_mrt',
        'floor_area_sqm', 'min_dist_cbd', 'flat_type_1 ROOM', 'flat_type_2 ROOM', 
        'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM', 'flat_type_EXECUTIVE', 
        'flat_type_MULTI-GENERATION']
 
X = df_model[exog]
y = df_model['adjusted_resale_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#%pip install xgboost

In [5]:
"""1. get a bse set of params for the train set
    use randomsearch to iterate thru the possible params"""

xgb_model = xgb.XGBRegressor(objective = "reg:squarederror") ## benchmark model 

bst_final=tune(xgb_model, X_train,y_train,
                                   X_test,y_test,
                                   model_type="xgboost",
                                   n_iter=20, 
                                   cv=5, 
                                   verbose=1, 
                                   n_jobs=-1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:08:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:08:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:08:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:08:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/fytia

[0]	eval-rmse:475715.49988
[1]	eval-rmse:383402.65735
[2]	eval-rmse:309816.66205
[3]	eval-rmse:251726.65912
[4]	eval-rmse:204854.72604


/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:17:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


[5]	eval-rmse:168078.33737
[6]	eval-rmse:139007.87407
[7]	eval-rmse:116109.88567
[8]	eval-rmse:97941.40315
[9]	eval-rmse:83287.97037
[10]	eval-rmse:71813.48484
[11]	eval-rmse:63457.85590
[12]	eval-rmse:57304.99464
[13]	eval-rmse:52876.34137
[14]	eval-rmse:49754.09971
[15]	eval-rmse:47148.94723
[16]	eval-rmse:45095.65985
[17]	eval-rmse:43821.56008
[18]	eval-rmse:42969.63860
[19]	eval-rmse:42232.72637
[20]	eval-rmse:41688.26047
[21]	eval-rmse:41509.79846
[22]	eval-rmse:41339.91328
[23]	eval-rmse:41142.72298
[24]	eval-rmse:41235.52541
[25]	eval-rmse:40866.76481
[26]	eval-rmse:40813.27096
[27]	eval-rmse:40506.38218
[28]	eval-rmse:40263.85043
[29]	eval-rmse:40243.97711
[30]	eval-rmse:40101.36187
[31]	eval-rmse:40072.85913
[32]	eval-rmse:39866.25904
[33]	eval-rmse:39519.68005
[34]	eval-rmse:39342.50939
[35]	eval-rmse:39219.88724
[36]	eval-rmse:38911.37441
[37]	eval-rmse:38813.55033
[38]	eval-rmse:38623.07985
[39]	eval-rmse:38505.89254
[40]	eval-rmse:38335.78293
[41]	eval-rmse:38267.54009
[42

In [ ]:
bst_final.save_model("models/xgb_model.bin") # Change to the proper directory


/Users/fytian/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:18:46] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [4]:
## compare with randomforrest regressors

rf=RandomForestRegressor(random_state=42)
random_forest=tune(rf, X_train,y_train,
                                   X_test,y_test,
                                   model_type="random_forest",
                                   n_iter=20, 
                                   cv=5, 
                                   verbose=1, 
                                   n_jobs=-1)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/fytian/anaconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


## After hypertuning the two model, we compare the performance of RF vs XGBOOOST using RMSE, MAPE and %overprediction

In [10]:
from sklearn.metrics import r2_score, mean_squared_error
y_pred=random_forest.predict(X_test)
y_insample_pred=random_forest.predict(X_train)
tolerance = 0.05 * y_test
underprice_pct = ((y_test - y_pred) > tolerance).mean() 
print(f"% Underpricing: {underprice_pct * 100:.2f}%")

overprice_pct = ((y_pred - y_test) > tolerance).mean() 
print(f"% Overpricing: {overprice_pct * 100:.2f}%")


def mape (y_actual, y_predicted): 
    mape = np.mean(np.abs((y_actual - y_predicted)/y_actual))*100 
    return mape 

metrics = {
    "Metric": ["R² Score", "RMSE", "MAPE"],
    "RF Train": [round(r2_score(y_train, y_insample_pred), 3),
                  round(np.sqrt(mean_squared_error(y_train, y_insample_pred)), 4),
                  round(mape(y_train, y_insample_pred), 4)],
    "RF Test": [round(r2_score(y_test, y_pred), 3),
                 round(np.sqrt(mean_squared_error(y_test, y_pred)), 4),
                 round(mape(y_test, y_pred), 4)]
}

model_evaluation_df = pd.DataFrame(metrics)
print(model_evaluation_df)


% Underpricing: 17.36%
% Overpricing: 20.13%
     Metric    RF Train     RF Test
0  R² Score      0.9870      0.9610
1      RMSE  21442.4299  36645.7040
2      MAPE      2.8940      4.8657


In [11]:
from sklearn.metrics import r2_score, mean_squared_error
pred_xg_train = bst_final.predict(xgb.DMatrix(X_train)) 
pred_xg_test = bst_final.predict(xgb.DMatrix(X_test))  

# diff = y_test - pred_xg_test
# underprice_pct = (diff > 10000).mean()  # Only count when underpriced by more than 10k (30%)

# we only count it as underpricing if it is more than 5% below actual (14%)
tolerance = 0.05 * y_test
underprice_pct = ((y_test - pred_xg_test) > tolerance).mean() 
print(f"% Underpricing: {underprice_pct * 100:.2f}%")

overprice_pct = ((pred_xg_test - y_test) > tolerance).mean() 
print(f"% Overpricing: {overprice_pct * 100:.2f}%")

def mape (y_actual, y_predicted): 
    mape = np.mean(np.abs((y_actual - y_predicted)/y_actual))*100 
    return mape 

metrics = {
    "Metric": ["R² Score", "RMSE", "MAPE"],
    "XGB Train": [round(r2_score(y_train, pred_xg_train), 3),
                  round(np.sqrt(mean_squared_error(y_train, pred_xg_train)), 4),
                  round(mape(y_train, pred_xg_train), 4)],
    "XGB Test": [round(r2_score(y_test, pred_xg_test), 3),
                 round(np.sqrt(mean_squared_error(y_test, pred_xg_test)), 4),
                 round(mape(y_test, pred_xg_test), 4)]
}

model_evaluation_df = pd.DataFrame(metrics)
print(model_evaluation_df)

% Underpricing: 13.03%
% Overpricing: 21.89%
     Metric   XGB Train    XGB Test
0  R² Score      0.9940      0.9660
1      RMSE  13843.5232  34500.4440
2      MAPE      1.8377      4.5768


## We decided on using XGBOOST which is traiend using assymetric loss , such that it favor over prediction, and it have lower RMSE, We now perform conformal prediction on its predicted output.

In [8]:
"""3. This is the final step, our model predict the value + the conformal interval of alpha%, alpha can be adjusted"""


y_pred = bst_final.predict(xgb.DMatrix(X_test)) 

# STEP 3: Calculate residuals and quantile threshold q
residuals = np.abs(y_test - y_pred)

alpha = 0.1  # for 90% confidence
q = np.quantile(residuals, 1 - alpha) 

## this so FE can change alpha top get a diff  q

In [10]:
with open("q_value.txt", "w") as f:
    f.write(str(q))

In [14]:
"""This fucntion takes in our best model bst_final. and input from FE  and the q defined above
output the predicted price and the conformal_prediction of 90% interval
exog = ['remaining_lease', 'min_dist_sch', 'storey_median', 'min_dist_mrt',
        'floor_area_sqm', 'min_dist_cbd', 

        ##the below are 0 or 1 
        'flat_type_1 ROOM', 'flat_type_2 ROOM', 
        'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM', 'flat_type_EXECUTIVE', 
        'flat_type_MULTI-GENERATION' ]
"""

##important function for FE to use 
conformal_predict(bst_final,[56,0.63,4,0.54,145,10.16,0,0,1,0,0,0,0],q)

 Predicted: 1002331.75, 90% CI: [948780.12, 1055883.38]


(array([1002331.75], dtype=float32),
 array([948780.1], dtype=float32),
 array([1055883.4], dtype=float32))

In [ ]:
import matplotlib.pyplot as plt
residuals = pred_xg_test - y_test

plt.figure(figsize=(8, 6))
plt.hist(residuals, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='Zero Error')
plt.xlabel('Prediction Error (Residuals)')
plt.ylabel('Frequency')
plt.title('Distribution of Prediction Errors')
plt.legend()
plt.show()
